In [1]:
import coco_text
COCO_DATA='/home/sravya/data/muse/coco/coco2014/'
ct = coco_text.COCO_Text(COCO_DATA+ 'COCO_Text.json')

loading annotations into memory...
0:00:01.455737
creating index...
index created!


In [1]:
import tensorflow as tf
from tqdm import tqdm
from PIL import Image

import sys
sys.path.append("/home/sravya/git/models")
from object_detection.utils import dataset_util

def writeTF(ImgIds, dataType, out_path):
    skipped_boxes = 0
    size_mismatch = 0
    empty_images = 0
    skipped_files = 0

    path = '{}{}/'.format(COCO_DATA, dataType)
    writer = tf.python_io.TFRecordWriter(out_path)

    for imgId in tqdm(ImgIds):
        img = ct.loadImgs(imgId)[0]
        filename = img['file_name']
        pil_img = Image.open(path + filename)
        width, height = pil_img.size

        #Sanity check
        if(width != img['width'] or height != img['height']):
            size_mismatch += 1
            width = img['width']
            height = img['height']    

        #Sanity check
        if(height == 0 or width == 0):
            empty_images += 1
            continue

        annIds = ct.getAnnIds(imgId)
        anns = ct.loadAnns(annIds)

        xmins = []
        ymins = []
        xmaxs = []
        ymaxs = []
        classes_text = [] # List of string class name of bounding box (1 per box)
        classes = []

        for ann in anns:
            xmin = ann['bbox'][0]
            ymin = ann['bbox'][1]
            b_width = ann['bbox'][2]
            b_height = ann['bbox'][3]
            xmax = xmin + b_width
            ymax = ymin + b_height

            #Normalize
            xmin = xmin/width
            xmax = xmax/width
            ymin = ymin/height
            ymax = ymax/height

            if(xmin<1 and ymin<1 and xmax<1 and ymax<1):
                xmins.append(xmin)
                ymins.append(ymin)
                xmaxs.append(xmax)
                ymaxs.append(ymax)
                classes_text.append('Text'.encode('utf8'))
                classes.append(1)
            else:
                skipped_boxes +=1

        with tf.gfile.GFile((path + filename), 'rb') as fid:
            encoded_image_data = fid.read()
        filename = path + filename
        filename = filename.encode('utf8')
        image_format = b'jpg'

        if (len(xmins) != 0):
            tf_example = tf.train.Example(features=tf.train.Features(feature={
            'image/height': dataset_util.int64_feature(height),
            'image/width': dataset_util.int64_feature(width),
            'image/filename': dataset_util.bytes_feature(filename),
            'image/source_id': dataset_util.bytes_feature(filename),
            'image/encoded': dataset_util.bytes_feature(encoded_image_data),
            'image/format': dataset_util.bytes_feature(image_format),
            'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
            'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
            'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
            'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
            'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
            'image/object/class/label': dataset_util.int64_list_feature(classes),
            }))
            writer.write(tf_example.SerializeToString())        
        else:
            skipped_files += 1
    print('skipped_boxes:{} size_mismatch: {} empty_images: {} skipped_files: {}'.format(
        skipped_boxes, size_mismatch, empty_images, skipped_files))


In [33]:
training_record = '/home/sravya/data/muse/coco/coco2014/TF/training.record'
train_dir='train2014'
valid_record = '/home/sravya/data/muse/coco/coco2014/TF/valid.record'
#val_dir='val2014'

train_img_ids = ct.getImgIds(imgIds=ct.train, catIds=[('legibility','legible')])
valid_img_ids = ct.getImgIds(imgIds=ct.val, catIds=[('legibility','legible')])

In [34]:
writeTF(train_img_ids, train_dir, training_record)
writeTF(valid_img_ids, train_dir, valid_record)

  7%|▋         | 230/3522 [00:00<00:01, 2298.19it/s]

skipped_boxes:0 size_mismatch: 0 empty_images: 0 skipped_files: 0


100%|██████████| 3522/3522 [00:01<00:00, 2413.09it/s]

skipped_boxes:0 size_mismatch: 0 empty_images: 0 skipped_files: 0
